In [1]:
from keras.preprocessing import image
import numpy as np
from numpy.linalg import norm
import pickle
import os
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.models import Model
from keras.layers import (Dense,GlobalAveragePooling2D,Input)


Using TensorFlow backend.


In [2]:
import os
root_dir = 'caltech101'
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']
def get_file_list(dir):
    file_list = []
    counter = 1
    labels = []
    for root, directories, filenames in os.walk(dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
                counter += 1
                labels.append(root)
    return file_list,labels
file_list,labels=get_file_list(root_dir)

In [11]:
labels[10]

'caltech101/binocular'

In [14]:
l =pickle.load(open('data/filenames-caltech101.pickle', 'rb'))
len(l)

5124

In [5]:
model =MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224,3))
model.trainable=False
def feature_extraction(img_path, model):
    img = image.load_img(img_path, target_size=(224,224,3))
    img_array = image.img_to_array(img)
    expand_img = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expand_img)
    feature = model.predict(preprocessed_img)
    flatten = feature.flatten()
    norm_features = flatten/norm(flatten)
    return norm_features

In [6]:
input = Input(shape=(224,224,3))
x = model(input)
x =GlobalAveragePooling2D()(x)
x =Dense(512,activation=None)(x)
x =Dense(256, activation=None)(x)
x =Dense(40,activation='softmax')(x)
optimize_model =Model(inputs=input, outputs=x)
optimize_model.compile(loss='categorical_crossentropy',optimizer="adam", metrics=['accuracy'])

In [7]:
from keras.preprocessing.image import ImageDataGenerator
root_dir = 'caltech101'
batch_size = 16
nb_train_samples = 8124 
epochs = 8
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    root_dir,
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='categorical')

optimize_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs)


Found 8124 images belonging to 40 classes.
Epoch 1/8
507/507 [==============================] - 529s 1s/step - loss: 0.8760 - accuracy: 0.8213
Epoch 2/8
507/507 [==============================] - 528s 1s/step - loss: 0.6313 - accuracy: 0.8822
Epoch 3/8
507/507 [==============================] - 529s 1s/step - loss: 0.5917 - accuracy: 0.8974
Epoch 4/8
507/507 [==============================] - 530s 1s/step - loss: 0.5931 - accuracy: 0.9036
Epoch 5/8
507/507 [==============================] - 521s 1s/step - loss: 0.4958 - accuracy: 0.9155
Epoch 6/8
507/507 [==============================] - 1180s 2s/step - loss: 0.5546 - accuracy: 0.9128
Epoch 7/8
507/507 [==============================] - 530s 1s/step - loss: 0.5107 - accuracy: 0.9181
Epoch 8/8
507/507 [==============================] - 527s 1s/step - loss: 0.5476 - accuracy: 0.9256


In [8]:

optimize_model.layers.pop()
model1 = Model(optimize_model.input, optimize_model.layers[-1].output)
model1.save('feature_extraction_model.h5')

In [9]:
from keras.models import load_model
from tqdm import tqdm
model3 = load_model('feature_extraction_model.h5')
feature_list = []
for i in tqdm(range(len(file_list))):
    feature_list.append(feature_extraction(file_list[i], model3))
print(len(feature_list[0]))    

/home/ojay/anaconda3/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
100%|██████████| 8124/8124 [10:28<00:00, 12.93it/s]

256


In [16]:
pickle.dump(file_list, open('data/labels-caltech101.pickle', 'wb'))
pickle.dump(feature_list, open('data/features-caltech101.pickle', 'wb'))

In [ ]:
feature_list = pickle.load(open('data/increamentalpcafeatures-caltech101.pickle', 'rb'))
filenames = pickle.load(open('data/features/filenames-caltech101.pickle', 'rb'))

In [ ]:
from keras.models import load_model
model3 = load_model('similarity_model_improved.hdf5')
p = filenames[10]
print(p)
features = feature_extraction(model3,'caltech101/binocular/image_0008.jpg')